In [ ]:
# preparations for calculating watersheds on WIMS points

# Prepare the direction and flow accumulation layers based on the used DEM
out_surface_raster = arcpy.sa.Fill("IHDTM_50m_GB.tif", None); 
out_surface_raster.save("FilledDEM")
out_flow_direction_raster = arcpy.sa.FlowDirection("FilledDEM", "NORMAL", None, "D8"); 
out_flow_direction_raster.save("UKFlowDirection")
with arcpy.EnvManager(compression="'LZW' 75"):
    arcpy.management.CopyRaster("UKFlowDirection", "C:\Temp\UKFlowDirectionCalculated", '', None, "-999", "NONE", "NONE", "32_BIT_FLOAT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")
out_accumulation_raster = arcpy.sa.FlowAccumulation("UKFlowDirection", None, "FLOAT", "D8"); 
out_accumulation_raster.save("UKFlowAccumulation")
with arcpy.EnvManager(compression="'LZW' 75"):
    arcpy.management.CopyRaster("UKFlowAccumulation", "C:\Temp\UKFlowAccumulationCalculated", '', None, "-999", "NONE", "NONE", "32_BIT_FLOAT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [2]:
# Generate a stream raster from the accumulated layer, 
# stating that a stream needs to be an accumulation of 250 cells or more  
with arcpy.EnvManager(overwriteOutput=True):
    out_raster = arcpy.sa.SetNull("UKFlowAccumulationCalculated.tif", "UKFlowAccumulationCalculated.tif", "VALUE < 250"); 
    out_raster.save("SetNull_Rast")      
    out_raster = arcpy.sa.Int("SetNull_Rast"); 
    out_raster.save("Int_Rast")
    arcpy.conversion.RasterToPolyline("Int_Rast", "C:\Temp\RiverLinefromAccumulated.shp", "NODATA", 0, "NO_SIMPLIFY", "Value")

    # Read there should be a stream within 100 meters, else it will get code "-1"
    # else the point is considered as unclear or not matching (note we have enough points)
    arcpy.analysis.Near("AverageRiverPointsNearNFRA", "RiverLinefromAccumulated", "100 Meters", "LOCATION", "NO_ANGLE", "PLANAR", "NEAR_FID NEAR_FID2;NEAR_DIST NEAR_DIST2;NEAR_X NEAR_X2;NEAR_Y NEAR_Y2")
    # select all points Not having code -1 and save
    arcpy.management.SelectLayerByAttribute("AverageRiverPointsNearNFRA", "NEW_SELECTION", "NEAR_X2 > 0", None)
    arcpy.management.CopyFeatures("AverageRiverPointsNearNFRA", r"\AverageRiverPointswithin100", '', None, None, None)
    
    # Make a new point layer based on snapped points to estimated stream network based on the DEM
    arcpy.conversion.ExportTable("AverageRiverPointswithin100", r"AverageRiverPoin_ExportTable", '', "NOT_USE_ALIAS", 'F_IDNumber "F_IDNumber" true true false 4 Long 0 0,First,#,AverageRiverPointswithin100,F_IDNumber,-1,-1;F_Sampling "F_Sampling" true true false 254 Text 0 0,First,#,AverageRiverPointswithin100,F_Sampling,0,254;F_Name_ "F_Name_" true true false 254 Text 0 0,First,#,AverageRiverPointswithin100,F_Name_,0,254;F_Determin "F_Determin" true true false 254 Text 0 0,First,#,AverageRiverPointswithin100,F_Determin,0,254;F_Determ_1 "F_Determ_1" true true false 4 Long 0 0,First,#,AverageRiverPointswithin100,F_Determ_1,-1,-1;F_Determ_2 "F_Determ_2" true true false 254 Text 0 0,First,#,AverageRiverPointswithin100,F_Determ_2,0,254;F_FlowType "F_FlowType" true true false 254 Text 0 0,First,#,AverageRiverPointswithin100,F_FlowType,0,254;F_Purpose_ "F_Purpose_" true true false 254 Text 0 0,First,#,AverageRiverPointswithin100,F_Purpose_,0,254;F_SampleEa "F_SampleEa" true true false 4 Long 0 0,First,#,AverageRiverPointswithin100,F_SampleEa,-1,-1;F_SampleNo "F_SampleNo" true true false 4 Long 0 0,First,#,AverageRiverPointswithin100,F_SampleNo,-1,-1;F_Median_ "F_Median_" true true false 8 Double 0 0,First,#,AverageRiverPointswithin100,F_Median_,-1,-1;F_Uncertai "F_Uncertai" true true false 8 Double 0 0,First,#,AverageRiverPointswithin100,F_Uncertai,-1,-1;F_NrSample "F_NrSample" true true false 4 Long 0 0,First,#,AverageRiverPointswithin100,F_NrSample,-1,-1;NrYears "NrYears" true true false 4 Long 0 0,First,#,AverageRiverPointswithin100,NrYears,-1,-1;NEAR_FID "NEAR_FID" true true false 4 Long 0 0,First,#,AverageRiverPointswithin100,NEAR_FID,-1,-1;NEAR_DIST "NEAR_DIST" true true false 8 Double 0 0,First,#,AverageRiverPointswithin100,NEAR_DIST,-1,-1;NEAR_X "NEAR_X" true true false 8 Double 0 0,First,#,AverageRiverPointswithin100,NEAR_X,-1,-1;NEAR_Y "NEAR_Y" true true false 8 Double 0 0,First,#,AverageRiverPointswithin100,NEAR_Y,-1,-1;NEAR_FID2 "NEAR_FID2" true true false 4 Long 0 0,First,#,AverageRiverPointswithin100,NEAR_FID2,-1,-1;NEAR_DIST2 "NEAR_DIST2" true true false 8 Double 0 0,First,#,AverageRiverPointswithin100,NEAR_DIST2,-1,-1;NEAR_X2 "NEAR_X2" true true false 8 Double 0 0,First,#,AverageRiverPointswithin100,NEAR_X2,-1,-1;NEAR_Y2 "NEAR_Y2" true true false 8 Double 0 0,First,#,AverageRiverPointswithin100,NEAR_Y2,-1,-1', None)
    arcpy.management.XYTableToPoint("AverageRiverPoin_ExportTable", r"AverageRiverPoin_ExportTable_XYTableToPoint", "NEAR_X2", "NEAR_Y2", None, 'PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",-100000.0],PARAMETER["Central_Meridian",-2.0],PARAMETER["Scale_Factor",0.9996012717],PARAMETER["Latitude_Of_Origin",49.0],UNIT["Meter",1.0]];-5220400 -15524400 450481592.767097;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision')
    arcpy.management.CopyFeatures("AverageRiverPoin_ExportTable_XYTableToPoint", r"RiverPointsSelected", '', None, None, None)
   # add an unique indentifier
    arcpy.conversion.ExportFeatures("RiverPointsSelected", r"C:\DataDanny\AgLand SDR\Validation\AverageRiverPointsSnappedNFRA100.shp", '', "NOT_USE_ALIAS", 'F_IDNumber "F_IDNumber" true true false 4 Long 0 0,First,#,RiverPointsSelected,F_IDNumber,-1,-1;F_Sampling "F_Sampling" true true false 254 Text 0 0,First,#,RiverPointsSelected,F_Sampling,0,254;F_Name_ "F_Name_" true true false 254 Text 0 0,First,#,RiverPointsSelected,F_Name_,0,254;F_Determin "F_Determin" true true false 254 Text 0 0,First,#,RiverPointsSelected,F_Determin,0,254;F_Determ_1 "F_Determ_1" true true false 4 Long 0 0,First,#,RiverPointsSelected,F_Determ_1,-1,-1;F_Determ_2 "F_Determ_2" true true false 254 Text 0 0,First,#,RiverPointsSelected,F_Determ_2,0,254;F_FlowType "F_FlowType" true true false 254 Text 0 0,First,#,RiverPointsSelected,F_FlowType,0,254;F_Purpose_ "F_Purpose_" true true false 254 Text 0 0,First,#,RiverPointsSelected,F_Purpose_,0,254;F_SampleEa "F_SampleEa" true true false 4 Long 0 0,First,#,RiverPointsSelected,F_SampleEa,-1,-1;F_SampleNo "F_SampleNo" true true false 4 Long 0 0,First,#,RiverPointsSelected,F_SampleNo,-1,-1;F_Median_ "F_Median_" true true false 8 Double 0 0,First,#,RiverPointsSelected,F_Median_,-1,-1;F_Uncertai "F_Uncertai" true true false 8 Double 0 0,First,#,RiverPointsSelected,F_Uncertai,-1,-1;F_NrSample "F_NrSample" true true false 4 Long 0 0,First,#,RiverPointsSelected,F_NrSample,-1,-1;NrYears "NrYears" true true false 4 Long 0 0,First,#,RiverPointsSelected,NrYears,-1,-1;NEAR_FID "NEAR_FID" true true false 4 Long 0 0,First,#,RiverPointsSelected,NEAR_FID,-1,-1;NEAR_DIST "NEAR_DIST" true true false 8 Double 0 0,First,#,RiverPointsSelected,NEAR_DIST,-1,-1;NEAR_X "NEAR_X" true true false 8 Double 0 0,First,#,RiverPointsSelected,NEAR_X,-1,-1;NEAR_Y "NEAR_Y" true true false 8 Double 0 0,First,#,RiverPointsSelected,NEAR_Y,-1,-1;NEAR_FID2 "NEAR_FID2" true true false 4 Long 0 0,First,#,RiverPointsSelected,NEAR_FID2,-1,-1;NEAR_DIST2 "NEAR_DIST2" true true false 8 Double 0 0,First,#,RiverPointsSelected,NEAR_DIST2,-1,-1;NEAR_X2 "NEAR_X2" true true false 8 Double 0 0,First,#,RiverPointsSelected,NEAR_X2,-1,-1;NEAR_Y2 "NEAR_Y2" true true false 8 Double 0 0,First,#,RiverPointsSelected,NEAR_Y2,-1,-1;DH_ID "DH_ID" true true false 512 Text 0 0,First,#,RiverPointsSelected,DH_ID,0,512', None)
    arcpy.management.CalculateField("AverageRiverPointsSnappedNFRA100", "DH_ID", "!FID!+1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
    

In [6]:
# Create watersheds for selected pour points as loops
import time
store_buffs = "AverageRiverPointsSnappedNFRA100" 
expression= " DH_ID > -1"
time_needed = []
with arcpy.da.SearchCursor(store_buffs, ["DH_ID"], where_clause=expression) as cursor: 
    for row in cursor:   
        # Name and select Shed
        exp = '"DH_ID" = ' + str(row[0])  
        with arcpy.EnvManager(overwriteOutput=True):
            start_time = time.time()
            Message = "Doing ID Shed: " + str(row[0]) + "   " + "Time Needed of last Shed: " + str(time_needed) + "   "
            print(Message, end="\r") 
            arcpy.analysis.Select(store_buffs,"TestedShed", exp)
            out_raster = arcpy.sa.SnapPourPoint("TestedShed", "UKFlowAccumulationCalculated.tif", 100, "DH_ID"); 
            out_raster.save("SnapPoint")  
            # The pour points are already snapped!, see preparation codes
            #out_raster = arcpy.sa.Watershed("UKFlowDirectionCalculated.tif", "TestedShed", "DH_ID");
            #with arcpy.EnvManager(cellSize = 10):
            out_raster = arcpy.sa.Watershed("UKFlowDirectionCalculated.tif", "SnapPoint", "Value");
            out_raster.save("WaterShedRast")
            arcpy.conversion.RasterToPolygon("WaterShedRast", "WaterShedPoly", "SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)
            arcpy.management.AddField("WaterShedPoly", "DH_ID", "LONG", 10, None, None, '', "NULLABLE", "NON_REQUIRED", '')
            arcpy.management.CalculateField("WaterShedPoly", "DH_ID", row[0], "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
            arcpy.management.JoinField("WaterShedPoly", "DH_ID", "TestedShed", "DH_ID", "F_Sampling;F_Name_;F_Determin;F_Determ_1;F_Determ_2;F_FlowType;F_Purpose_;F_Median_;F_Uncertai;F_NrSample;NrYears;F_SampleEa;F_SampleNo;NEAR_FID;NEAR_DIST;NEAR_DIST2;NEAR_X2;NEAR_Y2", "NOT_USE_FM", None)
            arcpy.management.JoinField("WaterShedPoly", "NEAR_FID", "NFRAStadions", "FID", "gdf_mean_f", "NOT_USE_FM", None)
            arcpy.management.AddField("WaterShedPoly", "Size_ha","DOUBLE", 18, 2 , None, '', "NULLABLE", "NON_REQUIRED", '')
            arcpy.management.CalculateGeometryAttributes("WaterShedPoly", "Size_ha AREA", '', "HECTARES", 'PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",-100000.0],PARAMETER["Central_Meridian",-2.0],PARAMETER["Scale_Factor",0.9996012717],PARAMETER["Latitude_Of_Origin",49.0],UNIT["Meter",1.0]]', "SAME_AS_INPUT")        
            # OutFile = "C:\Temp\WimsShed_" + str(row[0])
            arcpy.management.CopyFeatures("WaterShedPoly", "C:\Temp\WaterShedPoly", '', None, None, None)
            arcpy.management.Merge(r"C:\Temp\WaterShedPoly.shp;C:\Temp\WimsShed_Merge.shp", "WinShedtmp", r'Id "Id" true true false 10 Long 0 10,First,#,C:\Temp\WaterShedPoly.shp,Id,-1,-1,C:\Temp\WimsShed_Merge.shp,Id,-1,-1;gridcode "gridcode" true true false 10 Long 0 10,First,#,C:\Temp\WaterShedPoly.shp,gridcode,-1,-1,C:\Temp\WimsShed_Merge.shp,gridcode,-1,-1;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,C:\Temp\WaterShedPoly.shp,Shape_Leng,-1,-1,C:\Temp\WimsShed_Merge.shp,Shape_Leng,-1,-1;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,C:\Temp\WaterShedPoly.shp,Shape_Area,-1,-1,C:\Temp\WimsShed_Merge.shp,Shape_Area,-1,-1;DH_ID "DH_ID" true true false 10 Long 0 10,First,#,C:\Temp\WaterShedPoly.shp,DH_ID,-1,-1,C:\Temp\WimsShed_Merge.shp,DH_ID,-1,-1;F_Sampling "F_Sampling" true true false 254 Text 0 0,First,#,C:\Temp\WaterShedPoly.shp,F_Sampling,0,254,C:\Temp\WimsShed_Merge.shp,F_Sampling,0,254;F_Name_ "F_Name_" true true false 254 Text 0 0,First,#,C:\Temp\WaterShedPoly.shp,F_Name_,0,254,C:\Temp\WimsShed_Merge.shp,F_Name_,0,254;F_Determin "F_Determin" true true false 254 Text 0 0,First,#,C:\Temp\WaterShedPoly.shp,F_Determin,0,254,C:\Temp\WimsShed_Merge.shp,F_Determin,0,254;F_Determ_1 "F_Determ_1" true true false 10 Long 0 10,First,#,C:\Temp\WaterShedPoly.shp,F_Determ_1,-1,-1,C:\Temp\WimsShed_Merge.shp,F_Determ_1,-1,-1;F_Determ_2 "F_Determ_2" true true false 254 Text 0 0,First,#,C:\Temp\WaterShedPoly.shp,F_Determ_2,0,254,C:\Temp\WimsShed_Merge.shp,F_Determ_2,0,254;F_FlowType "F_FlowType" true true false 254 Text 0 0,First,#,C:\Temp\WaterShedPoly.shp,F_FlowType,0,254,C:\Temp\WimsShed_Merge.shp,F_FlowType,0,254;F_Purpose_ "F_Purpose_" true true false 254 Text 0 0,First,#,C:\Temp\WaterShedPoly.shp,F_Purpose_,0,254,C:\Temp\WimsShed_Merge.shp,F_Purpose_,0,254;F_Median_ "F_Median_" true true false 19 Double 0 0,First,#,C:\Temp\WaterShedPoly.shp,F_Median_,-1,-1,C:\Temp\WimsShed_Merge.shp,F_Median_,-1,-1;F_Uncertai "F_Uncertai" true true false 19 Double 0 0,First,#,C:\Temp\WaterShedPoly.shp,F_Uncertai,-1,-1,C:\Temp\WimsShed_Merge.shp,F_Uncertai,-1,-1;F_NrSample "F_NrSample" true true false 10 Long 0 10,First,#,C:\Temp\WaterShedPoly.shp,F_NrSample,-1,-1,C:\Temp\WimsShed_Merge.shp,F_NrSample,-1,-1;NrYears "NrYears" true true false 10 Long 0 10,First,#,C:\Temp\WaterShedPoly.shp,NrYears,-1,-1,C:\Temp\WimsShed_Merge.shp,NrYears,-1,-1;F_SampleEa "F_SampleEa" true true false 10 Long 0 10,First,#,C:\Temp\WaterShedPoly.shp,F_SampleEa,-1,-1,C:\Temp\WimsShed_Merge.shp,F_SampleEa,-1,-1;F_SampleNo "F_SampleNo" true true false 10 Long 0 10,First,#,C:\Temp\WaterShedPoly.shp,F_SampleNo,-1,-1,C:\Temp\WimsShed_Merge.shp,F_SampleNo,-1,-1;NEAR_FID "NEAR_FID" true true false 10 Long 0 10,First,#,C:\Temp\WaterShedPoly.shp,NEAR_FID,-1,-1,C:\Temp\WimsShed_Merge.shp,NEAR_FID,-1,-1;NEAR_DIST "NEAR_DIST" true true false 19 Double 0 0,First,#,C:\Temp\WaterShedPoly.shp,NEAR_DIST,-1,-1,C:\Temp\WimsShed_Merge.shp,NEAR_DIST,-1,-1;NEAR_DIST2 "NEAR_DIST2" true true false 19 Double 0 0,First,#,C:\Temp\WaterShedPoly.shp,NEAR_DIST2,-1,-1,C:\Temp\WimsShed_Merge.shp,NEAR_DIST2,-1,-1;NEAR_X2 "NEAR_X2" true true false 19 Double 0 0,First,#,C:\Temp\WaterShedPoly.shp,NEAR_X2,-1,-1,C:\Temp\WimsShed_Merge.shp,NEAR_X2,-1,-1;NEAR_Y2 "NEAR_Y2" true true false 19 Double 0 0,First,#,C:\Temp\WaterShedPoly.shp,NEAR_Y2,-1,-1,C:\Temp\WimsShed_Merge.shp,NEAR_Y2,-1,-1;gdf_mean_f "gdf_mean_f" true true false 19 Double 0 0,First,#,C:\Temp\WaterShedPoly.shp,gdf_mean_f,-1,-1,C:\Temp\WimsShed_Merge.shp,gdf_mean_f,-1,-1;Size_ha "Size_ha" true true false 19 Double 0 0,First,#,C:\Temp\WaterShedPoly.shp,Size_ha,-1,-1,C:\Temp\WimsShed_Merge.shp,Size_ha,-1,-1', "NO_SOURCE_INFO")
            arcpy.management.CopyFeatures("WinShedtmp", "C:\Temp\WimsShed_Merge.shp", '', None, None, None)
        f = ["SnapPoint","TestedShed","WaterShedRast","WaterShedPoly","WinShedtmp"]
        for x in f:
            arcpy.Delete_management(x)
        time_needed = (time.time() - start_time)  

In [2]:
# Code for extracting from Sediment export rasters based on Wims points near NFRA sites.
# Potentially overlapping so written as loops.
import arcpy
import numpy  
import csv  
arcpy.SetLogHistory(False)
store_buffs = "WaterShedsNearNFRASnapped100" 
expression=" DH_ID > -1"
time_needed = []
with arcpy.da.SearchCursor(store_buffs, ["DH_ID"], where_clause=expression) as cursor: 
  with arcpy.EnvManager(cellSize = 10):
    for row in cursor:   
          start_time = time.time()
          
        # Name and select Shed
          exp = '"DH_ID" = ' + str(row[0])  
          with arcpy.EnvManager(overwriteOutput=True):
              arcpy.analysis.Select(store_buffs, "TestedShed", exp)
          Message = "Doing ID Shed: " + str(row[0]) + "   " + "Time Needed of last Shed: " + str(time_needed) + "   "
          print(Message, end="\r") 
          for x in range(0,4):
             if x == 0:
                 density_ras = "sed_export_GEAR_FullOnce100meter.tif" 
                 OutBase = "C:/Temp/GEAR_100m_Export_"
                 Zonalrast = "GEAR_100m_Export"
             elif x == 1:
                 density_ras = "usle_GEAR_FullOnce100meter.tif" 
                 OutBase = "C:/Temp/GEAR_100m_USLE_"
                 Zonalrast = "GEAR_100m_USLE"
             elif x == 2:
                 density_ras = "usle_JRC_FullOnce.tif" 
                 OutBase = "C:/Temp/JRC_Full_USLE_"
                 Zonalrast = "JRC_Full_USLE"       
             elif x == 3:
                 density_ras = "usle_GEAR_FullOnce.tif" 
                 OutBase = "C:/Temp/GEAR_Full_USLE_"
                 Zonalrast = "GEAR_Full_USLE"    
             else:
                 print("This code stops")
             # Extract Data
             with arcpy.EnvManager(overwriteOutput=True):
                 arcpy.sa.ZonalStatisticsAsTable("TestedShed", "OBJECTID", density_ras, Zonalrast ,"DATA", "ALL")
                 # Write Data
                 table = Zonalrast
                 arcpy.management.AddField(table, "DH_ID", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
                 arcpy.management.CalculateField(table, "DH_ID", row[0], "PYTHON3", '', "LONG")
                 arcpy.management.JoinField(table, "DH_ID", "TestedShed", "DH_ID", "Size_ha", "NOT_USE_FM", None)
                 arcpy.management.JoinField(table, "DH_ID", "TestedShed", "DH_ID", "gdf_mean_f", "NOT_USE_FM", None)
                 arcpy.management.JoinField(table, "DH_ID", "TestedShed", "DH_ID", "NEAR_FID", "NOT_USE_FM", None) 
                 arcpy.management.JoinField(table, "DH_ID", "TestedShed", "DH_ID", "F_Median_", "NOT_USE_FM", None)
                 arcpy.management.JoinField(table, "DH_ID", "TestedShed", "DH_ID", "F_Uncertai", "NOT_USE_FM", None)
                 arcpy.management.JoinField(table, "DH_ID", "TestedShed", "DH_ID", "F_NrSample", "NOT_USE_FM", None)
                 arcpy.management.JoinField(table, "DH_ID", "TestedShed", "DH_ID", "NrYears", "NOT_USE_FM", None) 
                 outfile = OutBase + str(row[0])+ ".csv"
                 fields = arcpy.ListFields(table)  
                 field_names = [field.name for field in fields]  
                 with open(outfile,'w') as f:  
                     w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                     #--write all field names to the output file  
                     w.writerow(field_names)  
                     for mow in arcpy.SearchCursor(table):  
                        field_vals = [mow.getValue(field.name) for field in fields]  
                        w.writerow(field_vals) 
          time_needed = (time.time() - start_time)              
          arcpy.Delete_management(Zonalrast)
          arcpy.Delete_management("TestedShed")  